In [1]:
import pandas as pd
import pickle
import os
import numpy as np
import random
from colog.colog import colog
from datetime import datetime, timedelta
from tqdm import tqdm
c = colog()

In [2]:
a =1 

In [3]:
def load_data_one_stock(path):
  load_data = pd.DataFrame()
  try:
    file = open(path, 'rb')
    load_data= pickle.load(file)
    file.close()
  except Exception as e:
    print(e)
  return load_data

In [4]:
# SETTINGS
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
# stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']

# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA'] + ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']

folder_saved_models = f'historical_data\\period2yinterval1h'
parent_path = os.getcwd()
buy_money = 2000
commission_value = 0.0005
trainig_part = 0.5
test_part = 0.5

In [5]:
import numpy as np
np.amax([1,2,4,6,-6])

np.int64(6)

Exlude companies with bad ration and retraining!!!

Currten best settings:
{'is_near_global_max_prt': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.2, 'buy_ratio_border': 10}
-3458.0764813327787 495


In [6]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
   gmin = min(df['close'].iloc[i - k: i])
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
   gmin = min(df['close'].iloc[0: i])
  # if gmax == reference_point:
  #   result = False
  # elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
  #   result = True
  # else:
  #   result = False
  result = 100 * (df['close'].iloc[i] - gmin) / (gmax - gmin) > prt

  return result

def global_max_prop(i, k=40):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   gmin = min(df['close'].iloc[i - k: i])
  else:
   gmax = max(df['close'].iloc[0: i])
   gmin = min(df['close'].iloc[0: i])

  return gmax / gmin

def number_red_candles(df, i, k=11):

  if i < k:
    number_red_candles = (df['ha_colour'][0 : i] == 'red').sum()
  else:
    number_red_candles = (df['ha_colour'][i - k : i] == 'red').sum()
  return number_red_candles

def number_green_candles(df, i, k=11):


  if i < k:
    number_green_candles = (df['ha_colour'][0 : i] == 'green').sum()
  else:
    number_green_candles = (df['ha_colour'][i - k : i] == 'green').sum()
  return number_green_candles

def more_than_pct(value1, value2, pct):
  if 100 * (value1 / value2 - 1) > pct:
    return True
  else:
    return False

def maximum(df, i, k=20):  
  return np.amax([df['close'].iloc[i -k : i].max(), df['open'].iloc[i - k : i].max(), df['close'].iloc[i], df['open'].iloc[i]])

def miminum(df, i, k=20):  
  return np.amin([df['close'].iloc[i - k : i].min(), df['open'].iloc[i - k : i].min(), df['close'].iloc[i], df['open'].iloc[i]])

def norm(df, i, k=20):
  min = miminum(df,i,k)
  max = maximum(df,i,k)
  return (df['close'].iloc[-1] - min) / (max - min + 0.0001)

def get_amps(df, i):
  amps_7 = []
  amps_30 = []
  amps_100 = []
  if i > 300:
    for k in range(i - 300, i):
      min = miminum(df, k, 7)
      max = maximum(df, k, 7)
      amps_7.append(max / min)
      min = miminum(df, k, 30)
      max = maximum(df, k, 30)
      amps_30.append(max / min)
      min = miminum(df, k, 100)
      max = maximum(df, k, 100)
      amps_100.append(max / min)
    
    amp_7 = np.sum(amps_7) / len(amps_7)
    amp_30 = np.sum(amps_30) / len(amps_30)
    amp_100 = np.sum(amps_100) / len(amps_100)
  else:
    amp_7 = 0
    amp_30 = 0
    amp_100 = 0

  return amp_7, amp_30, amp_100


In [7]:
amps_7, amps_30, amps_100 = {}, {}, {}

for ticker in tqdm(stock_name_list):
# for ticker in ['GOOG']:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  amp_7, amp_30, amp_100 = get_amps(df, df.shape[0])
  amps_7[ticker] = amp_7
  amps_30[ticker] = amp_30
  amps_100[ticker] = amp_100




 74%|███████▍  | 140/188 [00:30<00:07,  6.18it/s]

[Errno 2] No such file or directory: 'f:\\Git Projects\\HeikenAshiInvestingModel\\historical_data\\period2yinterval1h\\df_stock_PXD_period2y_interval1h.pkl'


100%|██████████| 188/188 [00:40<00:00,  4.69it/s]


In [24]:
test_list = []
for ticker in tqdm(stock_name_list):
  try:
    if amps_7[ticker] >= 1.02:
      # print(f'{ticker}, {amps_7[ticker]}')
      test_list.append(ticker)
  except:
    pass
print(test_list)

100%|██████████| 190/190 [00:00<00:00, 771459.59it/s]

['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD', 'ORCL', 'TXN', 'WFC', 'INTC', 'INTU', 'QCOM', 'BMY', 'CAT', 'BA', 'ISRG', 'GE', 'AMAT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'GS', 'AXP', 'ADI', 'C', 'CVS', 'SCHW', 'LRCX', 'SLB', 'ETN', 'PYPL', 'PANW', 'MU', 'SNPS', 'APD', 'KLAC', 'CDNS', 'TGT', 'FCX', 'F', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PH', 'MPC', 'APH', 'FTNT', 'MCHP', 'USB', 'JCI', 'PSX', 'ADSK', 'TDG', 'TT', 'EL', 'PCAR', 'OXY', 'TFC', 'CARR', 'IDXX', 'ON', 'COF', 'MNST', 'NUE', 'AIG', 'VLO', 'MRNA', 'ANET', 'IQV', 'CHTR', 'BIIB', 'DHI', 'ROK']


In [119]:
ticker = 'AMD'
# ticker = 'UNH'
# ticker = 'ADBE'
ticker = 'GOOG'
amps_7[ticker], amps_30[ticker], amps_100[ticker]
# dv = np.amin([amps_7[ticker] - 1, amps_30[ticker] - 1, amps_100[ticker] - 1]) - 0.001


(np.float64(1.02349316009157),
 np.float64(1.0554166780396679),
 np.float64(1.1099826011538592))

In [213]:
def one_stock_simulation(df, ticker, mode, _gain_coef = 1.005, _lose_coef = 0.97, print_sim_results=False, print_trans_history=False):
  '''
    Parameters:
      mode: train / full / test

    Returns:
      profits, wins, loses 
  '''

  history_log = {'number success': 0,
                'number fails': 0,
                'number wins': 0,
                'proportion': 0,
                'profit': 0,
                'transactions': [] }  
  
  sell_finished = True
  buy_order_placed = False
  buy_price = 0
  
  if mode == 'train':
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'full': 
    range_ = range(2, df.shape[0] - 1)
  if mode == 'test':
    range_ = range(int(df.shape[0] * trainig_part) + 1, df.shape[0] - 1)

  buy_i = 0
  conditions = []
  CM930, CM1030, CM1130, CM1230, CM1330 = 10000, 10000, 10000 ,10000 ,10000
  
  for i in range_:

    profit = df['high'].iloc[i] / (buy_price + 0.0001)

    min_7  = miminum(df, i - 1, 7)
    min_30  = miminum(df, i - 1 , 30)
    min_100  = miminum(df, i - 1, 100)
    max_7  = maximum(df, i - 1, 7)
    max_30  = maximum(df, i - 1, 30)
    max_100  = maximum(df, i - 1, 100)
    max_300  = maximum(df, i - 1, 300)
    cv = df['close'].iloc[i - 1]
    dist_from_max_7 = max_7 / cv
    dist_from_max_30 = max_30 / cv
    dist_from_max_100 = max_100 / cv

    if i >= buy_i:

      if sell_finished == False and df['high'].iloc[i] / buy_price > _gain_coef:
        # sell_price = float(df['high'].iloc[i])
        sell_price = buy_price * _gain_coef
        history_log['number success'] += 1
        number_buy_stocks = int(buy_money / buy_price)
        history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
        history_log['number wins'] += 1
        history_log['transactions'].append({
          'type': 'success',
          'buy_price' : round(buy_price, 2),
          'sell_price': round(sell_price, 2), 
          'buy_time' : buy_time,
          'sell_time' : df['high'].index[i]
      })
        sell_finished = True
        buy_order_placed = False

      if sell_finished == False and df['low'].iloc[i] / buy_price < _lose_coef :
        sell_price = buy_price * _lose_coef
        sell_finished = True
        buy_order_placed = False
        number_buy_stocks = int(buy_money / buy_price)
        history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
        history_log['number fails'] += 1
        history_log['transactions'].append({
          'type': 'fail',
          'buy_price' : round(buy_price, 2),
          'sell_price': round(sell_price, 2),
          'buy_time' : buy_time,
          'sell_time' : df['low'].index[i],
        })

    condition = False
    # dv = 0.04
    dv = np.amin([amps_7[ticker] - 1, amps_30[ticker] - 1, amps_100[ticker] - 1]) - 0.001
    # print(amps_7[ticker] - dist_from_max_7)
    try:
      # if (amps_7[ticker] - dist_from_max_7) > dv  \
      #   and (amps_30[ticker] - dist_from_max_30) > dv \
      #   and (amps_100[ticker] - dist_from_max_100) > dv  \
      #   and df['close'].iloc[i - 1] / df['close'].iloc[i - 100] < 1 - (amps_7[ticker] - 1) \
      #   and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] < 1.008 \
      #   and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] > 1.001 \
      #   and df['close'].iloc[i - 1] / df['open'].iloc[i - 1] < 1.008 \
      #   and max_300 / df['open'].iloc[i - 1] > 1 + (amps_100[ticker] - 1) / 2:

        # and max_300 / df['open'].iloc[i - 1] > 1.08:
        # and max_300 / df['open'].iloc[i - 1] > tp:
        # and max_300 / df['open'].iloc[i - 1] < 1 + (amps_100[ticker] - 1):
        # and df['pct'].iloc[i - 7: i].sum() < tp:

      # if (amps_30[ticker] - dist_from_max_30) > 0.005 \
      #   and (amps_100[ticker] - dist_from_max_100) > 0.04 \
      #   and df['close'].iloc[i - 1] / df['close'].iloc[i - 100] < 0.94 \
      #   and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] < 1.006 \
      #   and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] > 1.0015 \
      #   and df['close'].iloc[i - 1] / df['open'].iloc[i - 1] < 1.006 \
      #   and max_300 / df['open'].iloc[i - 1] > 1 + (amps_100[ticker] - 1) / 2:

      if (amps_30[ticker] - dist_from_max_30) > tp1 \
        and (amps_100[ticker] - dist_from_max_100) > tp2 \
        and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] < tp4 \
        and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] > tp5 \
        and df['close'].iloc[i - 1] / df['open'].iloc[i - 1] < tp6 \
        and max_300 / df['open'].iloc[i - 1] > 1 + (amps_100[ticker] - 1) / 2:
        
              # and df['close'].iloc[i - 1] / df['close'].iloc[i - 100] < tp3 
        
          # and df['close'].iloc[i - 1] / df['close'].iloc[i - 100] < 1 - (amps_7[ticker] - 1) \
      
        condition = True
    except:
      condition = False

    if False:
      pass
      # conditions = []
      # c1 = False
      # if df.index[i].hour == 10 and df.index[i].minute == 30:
      #   CM930 = df['close'].iloc[i - 1]
      # #   c1 = df['close'].iloc[i - 1] / df['close'].iloc[i - 2] > 1.005 \
      # #       and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] < max_930_value_to_buy
      # # conditions.append(c1)

      # c2 = False
      # if df.index[i].hour == 11 and df.index[i].minute == 30:
      #   # i : 11hours, i - 1 : 10hours, i - 2: 9hours, i - 3 : 15hours
      #   CM1030 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   c2 = more_than_pct(CM1030, CM930, 0.21) \
      #     and  df['open'].iloc[i - 2] / df['close'].iloc[i - 2]  > 1.01 \
      #     and df['close'].iloc[i - 3] / df['close'].iloc[i - 2] < 1.002 \
      #     and df['close'].iloc[i - 3] / df['close'].iloc[i - 2] > 0.99
      # conditions.append(c2)
  
      # c31, c32 = False, False
      # if df.index[i].hour == 12 and df.index[i].minute == 30:
      #    # i : 12hours, i - 1 : 11hours, i - 2: 10hours, i - 3 : 9hours
      #   CM1130 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   # c31 = more_than_pct(CM1130, CM1030, 0.01) and more_than_pct(CM1130, CM930, 0.45) \
      #   c31 = more_than_pct(CM1130, CM1030, tp1) and more_than_pct(CM1030, CM930, tp2) \
      #     #  and  df['open'].iloc[i - 3] / df['close'].iloc[i - 3]  > tp2 \
      #     #  and  df['open'].iloc[i - 3] / df['close'].iloc[i - 3]  < tp2 + tp4
      #   # {'tp2': np.float64(0.992), 'tp4': np.float64(0.998)}
      #   c32 = more_than_pct(CM1130, CM1030, 0.01) and more_than_pct(CM930, CM1030, 0.01) and more_than_pct(CM1130, CM930, tp) # local minumum (1.6)
      # conditions.append(c31)
      # conditions.append(c32)
    
      # c41, c42 = False, False
      # if df.index[i].hour == 13 and df.index[i].minute == 30:
      #   CM1230 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   c41 = CM1230 > CM1130 and CM1230 > CM1030 and CM1230 > CM930
      #   # c42 = CM1230 > CM1130 and CM1230 < CM930

      #   c42 = more_than_pct(CM1230, CM1130, 0) and more_than_pct(CM930, CM1130, 0)  # local minumum 
      # conditions.append(c41)
      # conditions.append(c42)

      # c51, c52 = False, False
      # if df.index[i].hour == 14 and df.index[i].minute == 30:
      #   CM1330 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   c51 = CM1330 > CM1230 and CM1330 > CM1130 and CM1330 > CM1030 and CM1330 > CM930
      #   c52 = CM1330 > CM1230 and CM1230 < CM930
      # conditions.append(c51)
      # conditions.append(c52)

      # condition = False
      # if df.index[i].hour == 13 and df.index[i - 1].hour == 12:
      # # moomoom time: i: 14:30; i-1: 13:30: i-2: 12:30; i-3: 11:30; i-4: 10:30 i-5 : 16:00 : i-6: 16:00 
      # # candle is green
      # # candle pct more than 0 
      # # more taht close 9:30
      # # sum pct 10:30 - 12:30  more than 0 
      # # 9:30 less than -0l77
      #   if df['close'].iloc[i - 1] > df['open'].iloc[i - 1] \
      #     and df['pct'].iloc[i - 1] > 0 \
      #     and df['pct'].iloc[i - 4 : i - 1].sum() > 0\
      #     and df['close'].iloc[i - 1] > df['close'].iloc[i - 5] \
      #     and (df['pct'].iloc[i - 4] < -0.5 \
      #         or df['close'].iloc[i - 6] / df['close'].iloc[i - 5] > 0.996)\
      #     and norm(df, 7) < 0.8 \
      #     and norm(df, 220) > tp or False:
      #     condition = True
      
      # previos candle is green
      # condition = False
      # if df['close'].iloc[i - 1] > df['open'].iloc[i - 1] \
      #   and df['pct'].iloc[i - 1] > 0 \
      #   and norm(df, 7) > tp \
      #   and norm(df, 220) <= 1:
      #   condition = True

      # condition2 = False
      # CM1 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      # CM2 = (df['close'].iloc[i - 2] + df['open'].iloc[i - 2]) / 2
      # CM3 = (df['close'].iloc[i - 3] + df['open'].iloc[i - 3]) / 2
      # # if CM1 > CM2 and CM1 > CM3 and CM2 > CM3 \
      # if True \
      #   and df.index[i].hour != 15 \
      #   and df.index[i].hour != 9 \
      #   and df['close'].iloc[i - 4] < df['open'].iloc[i - 4]\
      #   and df['pct'].iloc[i - 1] > 0 \
      #   and df['pct'].iloc[i - 2] > 0 \
      #   and df['pct'].iloc[i - 3] > 0:
      #     condition2 = True

    if buy_order_placed == False \
      and condition:

      # buy_price = min(df['close'].iloc[i - 10: i].min(), df['open'].iloc[i - 10: i].min() )
      # buy_price = df['open'].iloc[i]
      buy_price = df['close'].iloc[i - 1]
      buy_time = df.index[i]
      
      # sell_finished = False
      # buy_i = i

      if df['low'].iloc[i] <= buy_price and df['high'].iloc[i] >= buy_price:
        buy_order_placed = True
        sell_finished = False
        buy_i = i
    
  history_log['proportion'] =   history_log['number wins'] / ( history_log['number fails'] + 1)

  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    print(f'amp_7 is {amps_7[ticker]}, amp_30 is {amps_30[ticker]}, amp_100 is {amps_100[ticker]},  ')
    print(history_log)

  if print_trans_history:
    for tran in history_log['transactions']:
      c.print(tran, color='blue')

  return history_log['profit'], history_log['number wins'], history_log['number fails']


In [220]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96
bull_trend_coef = 1.12
last_top_24hours_proportion_border = 1

# best_params = {'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 5, 'bull_trend_coef': 1.12}
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
# after last check
best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.5, 'RIV2': 0.9,
                'buy_ratio_border': 0, 'bull_trend_coef': 1.07, 'last_top_24hours_proportion_border': 1, 'nrc': 5, 'RIV1': 0.9, 'ni': 1, 
                'tp1': 0.005, 'tp2': 0.04, 'tp3': 1.001, 'tp4' : 1.001}
# best_params = {'tp1': np.float64(0.4), 'tp2': np.float64(1.001), 'tp3': np.float64(0.2), 'tp4': np.float64(1.001)}
# best_params = {'tp1': np.float64(0.2), 'tp2': np.float64(1.001), 'tp3': np.float64(0.2), 'tp4': np.float64(1.009)}
# best_params = {'tp1': np.float64(0.21000000000000002), 'tp2': np.float64(1.01), 'tp3': np.float64(0.2), 'tp4': np.float64(0.001)}
# best_params = {'tp1': 0.005, 'tp2': 0.04, 'tp3': 0.94, 'tp4': 1.006, 'tp5' : 1.0015, 'tp6' : 1.006}
best_params = {'tp1': np.float64(0.009), 'tp2': np.float64(0.075), 'tp3': np.float64(0.95), 
               'tp4': np.float64(1.004), 'tp5': 1.0015, 'tp6': np.float64(1.004)}
for param in best_params:
  vars()[param] = best_params[param]

# parameters_list_search = {
#   'is_near_global_max_prt' : [0, 20, 30, 50, 70, 80, 96, 100],
#   'distance_from_last_top': [0,2,3,5],
#   'last_top_ratio': [1, 1.005, 1.01, 1.05, 1.1],
#   'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25],
#   'buy_ratio_border' : [0,3,5,7,9,10],
#   'bull_trend_coef': [1.07, 1.12]
# }


parameters_list_search = {
  # 'tp1': np.arange(0, 0.01, 0.001),
  # 'tp2': np.arange(0.02, 0.09, 0.005),
  # 'tp3': np.arange(0.88, 1, 0.01),
  # 'tp4': np.arange(1.001, 1.009, 0.001),
  # 'tp5': [1.0015, 1.003, 1.005],
  # 'tp6': np.arange(1.001, 1.009, 0.001)

  # 'tp3': np.arange(0.92, 0.96, 0.01),
  # 'tp4': np.arange(1.002, 1.005, 0.001)

  'tp2': np.arange(0.02, 0.09, 0.005),
  'tp4': np.arange(1.002, 1.005, 0.001),
  'tp5': [1.0015, 1.003, 1.005],
  'tp6': np.arange(1.001, 1.009, 0.002)
}
# best_params = {'tp1': 0.005, 'tp2': 0.04, 'tp3': 0.94, 'tp4': 1.006, 'tp5' : 1.0015, 'tp6' : 1.006}

# {'is_near_global_max_prt': 70, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.2, 'buy_ratio_border': 7, 'bull_trend_coef': 1.07}

# parameters_list_search = {
#   # 'last_top_24hours_proportion_border': np.arange(1, 0, -0.1)
#   'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25]
# }

In [11]:
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']
len(stock_name_list)
max_930_value_to_buy = 3

In [245]:
# BEST PARAMETERS OPTIMIZATION  and OPTIMAL STOCK LIST
best_profit = -10**5
best_proportion = 0
result_improved = True
optimize_profit = True
optimize_proportion =  False
optimize_smart = True
ticker_stats = {}
stock_name_list_opt = stock_name_list[:]
# stock_name_list_opt = ['GOOG', 'XOM']
# stock_name_list_opt = ['AMD', 'AVGO']
# stock_name_list_opt = ['JPM']
# tp = 0.4
# stock_name_list_opt = ['AMD', 'AVGO', 'GOOG', 'XOM',
# 'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'LRCX', 'LLY',
# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE']
# stock_name_list_opt = ['BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'LRCX', 'LLY']

# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE']
# stock_name_list_opt = ['GOOG']
# stock_name_list_opt = ['UNH']

# stock_name_list_opt = ['DHI', 'CDNS', 'ORCL', 'CMG', 'ADBE', 'APH', 'AMD', 'NOW', 'EMR', 'ETN', 'EW', 'AMAT', 'FTNT', 'AIG', 'AVGO', 'TT', 'DE', 'GM', 'CAT', 'ON', 'INTU', 'LRCX', 'PYPL', 'LLY', 'GOOG', 'VLO', 'C', 'BAC', 'PH']

# stock_name_list_opt = ['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD']
# stock_name_list_opt = ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
#                        'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
#                        'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
#                        'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
#                        'BIIB','SPG','MSCI','DHI','ROK']
# profit > 0 
# stock_name_list_opt =['DHI', 'LRCX', 'CMG', 'GM', 'TGT', 'CDNS', 'CHTR', 'PANW', 'FTNT', 'HUM', 'PH', 'DXCM', 'FCX', 'AVGO', 'VLO', 'APH', 'BAC', 'IQV', 'NKE', 'PCAR', 'ROK', 'COF', 'C', 'GOOG', 'MS', 'CAT', 'NOW', 'GS', 'CARR', 'TFC', 'JCI', 'SLB', 'ON']
# profit < 0
# stock_name_list_opt = ['INTC', 'EMR', 'BA', 'USB', 'KLAC', 'EW', 'LLY', 'ETN', 'DE', 'MAR', 'TXN', 'PYPL', 'SBUX', 'AMD', 'EL', 'PNC', 'SCHW', 'ADI', 'NUE', 'WFC', 'ANET', 'QCOM', 'SNPS', 'F', 'AMAT', 'NXPI', 'BMY', 'MMM', 'MRNA', 'MCHP', 'MU']
# profit = 0 after first cycle
# stock_name_list_opt = ['ABT', 'LIN', 'MDT', 'NFLX', 'XOM', 'UNH', 'PG', 'MA', 'JNJ', 'V', 'CRM', 'CVX', 'PEP', 'MRK', 'ABBV',
#                         'ADBE', 'KO', 'COST', 'PFE', 'WMT', 'MCD', 'CSCO', 'ACN', 'TMO', 'HD', 'JPM', 'BIIB', 'WELL', 'CTAS',
#                         'AIG', 'ADM', 'O', 'KMB', 'TT', 'ADSK', 'UNP', 'COP', 'ISRG', 'IBM', 'GE', 'AMGN', 'SPGI', 'DHR', 
#                         'PM', 'INTU', 'UPS', 'VZ', 'LOW', 'HON', 'RTX', 'CMCSA', 'ORCL', 'NEE', 'DIS', 'MDLZ', 'ELV', 'T']

# stock_name_list_opt =['SYK', 'FDX', 'WM', 'PGR', 'APD', 'SHW', 'CME', 'REGN', 'SO', 'EOG', 'AON', 'ITW', 'CSX', 'BSX', 
#                           'EQIX', 'BDX', 'PLD', 'FI', 'MO', 'TMUS', 'MMC', 'AMT', 'CVS', 'VRTX', 'ADP', 'CI', 'ZTS', 'CB',
#                             'TJX', 'GILD', 'BLK', 'AXP', 'LMT', 'MSCI', 'SPG', 'STZ', 'HLT', 'WMB', 'EXC', 'D', 'AEP', 'MNST',
#                               'SRE', 'GIS', 'IDXX', 'ICE', 'NOC', 'HCA', 'CL', 'DUK', 'AZO', 'AJG', 'ECL', 'TEL', 'PSA', 'TDG',
#                                 'NSC', 'ROP', 'MPC', 'MCO', 'PSX', 'MSI', 'GD', 'CCI', 'MCK', 'OXY']
# stock_name_list_opt = ['ABT', 'LIN', 'MDT']
stock_name_list_opt = ['ON']

# for parameter in parameters_list_search:
while result_improved:
  for i in random.sample(range(len(parameters_list_search)), len(parameters_list_search)):
    parameter = list(parameters_list_search.keys())[i]
    total_profit_list = []
    total_loses_list = []
    total_proportion_list = []
    for value in parameters_list_search[parameter]:
      vars()[parameter] = value
      c.print(f'parametr is {parameter} value is {value}:', color='white')
      total_profit = 0
      total_loses = 0
      total_wins = 0

      for ticker in stock_name_list_opt:
        file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
        path = os.path.join(parent_path, folder_saved_models, file_name)
        df = load_data_one_stock(path)
        ticker_stats[ticker] = {
            'profit': -999,
            'wins': 0,
            'loses' : 0,
        }
        if df.shape[0] > 0:
          # profit, wins, loses= one_stock_simulation(df, ticker, mode='full')
          profit, wins, loses= one_stock_simulation(df, ticker,
                                                     mode='train',
                                                     _gain_coef=1.005,
                                                     _lose_coef=0.995)
          total_profit += profit
          total_loses += loses
          total_wins += wins
          ticker_stats[ticker] = {
            'profit': profit,
            'wins': wins,
            'loses' : loses,
          }
        else:
          stock_name_list_opt.remove(ticker)
      
      total_profit_list.append(total_profit)
      total_loses_list.append(total_loses)
      total_proportion_list.append(total_wins / (total_loses + 1))
      c.print(f'Total Profit is {total_profit:.2f}, total wins is {total_wins}, total loses is {total_loses} \
      proportion {total_wins/(total_loses + 1)}', color = 'red')
      # break if parametr is higher than previous value but profit decreased
      # if len(total_profit_list) > 1:
      #   if total_profit_list[-1] < total_profit_list[-2]:
      #     break
    
    if optimize_smart:
      if max(total_profit_list) > 0:
        optimize_proportion = False
        optimize_profit = True
      else:
        optimize_proportion = True
        optimize_profit = False

    c.print(f'total_profit_list: {total_profit_list}', color='orange')
    if optimize_profit:
      print('Profit optimization')
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
    elif optimize_proportion:
      print('Proportion optimization')
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_proportion_list)]
    else:
      vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    # np.max(total_profit_list), np.argmax(total_profit_list)

  parameters_list_best = {}
  for parameter in parameters_list_search:
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    
    parameters_list_best[parameter] = vars()[parameter]
    
  print(parameters_list_best)
  print(max(total_profit_list), min(total_loses_list))

  if optimize_smart:
    if max(total_profit_list) > 0:
      optimize_proportion = False
      optimize_profit = True
    else:
      optimize_proportion = True
      optimize_profit = False

  if optimize_proportion:
    print('Proportion optimization')
    result_improved = max(total_proportion_list) > best_proportion
    best_proportion = max(total_proportion_list) if result_improved else best_proportion
  else:
    print('Profit optimization')
    result_improved = max(total_profit_list) > best_profit
    best_profit = max(total_profit_list) if result_improved else best_profit
  
  if len(parameters_list_search) <=1:
    result_improved = False

  # Optimization which stocks to use:
  # for ticker in stock_name_list_opt:
  #   if ticker_stats[ticker]['wins'] / (ticker_stats[ticker]['loses'] + 1) < 3 \
  #       or ticker_stats[ticker]['profit'] < 0:
  #     stock_name_list_opt.remove(ticker)
  # c.print(f'Current optimal stock list len is {len(stock_name_list_opt)}, list is {stock_name_list_opt}', color='green')

parametr is tp2 value is 0.02:
Total Profit is 114.43, total wins is 43, total loses is 19       proportion 2.15
parametr is tp2 value is 0.025:
Total Profit is 102.72, total wins is 40, total loses is 18       proportion 2.1052631578947367
parametr is tp2 value is 0.030000000000000002:
Total Profit is 86.94, total wins is 38, total loses is 18       proportion 2.0
parametr is tp2 value is 0.035:
Total Profit is 82.87, total wins is 36, total loses is 17       proportion 2.0
parametr is tp2 value is 0.04000000000000001:
Total Profit is 94.56, total wins is 36, total loses is 16       proportion 2.1176470588235294
parametr is tp2 value is 0.045000000000000005:
Total Profit is 98.58, total wins is 35, total loses is 15       proportion 2.1875
parametr is tp2 value is 0.05:
Total Profit is 102.37, total wins is 34, total loses is 14       proportion 2.2666666666666666
parametr is tp2 value is 0.05500000000000001:
Total Profit is 98.50, total wins is 32, total loses is 13       proportion 

In [ ]:
print(stock_name_list_opt)
# len(stock_name_list_opt)

['GOOG', 'JPM', 'XOM', 'UNH', 'JNJ', 'V', 'AVGO', 'PG', 'LLY', 'MA', 'HD', 'CVX', 'MRK', 'PEP', 'COST', 'ABBV', 'ADBE', 'KO', 'CRM', 'WMT', 'MCD', 'CSCO', 'BAC', 'PFE', 'TMO', 'ACN', 'NFLX', 'ABT', 'AMD', 'LIN', 'ORCL', 'CMCSA', 'TXN', 'DIS', 'WFC', 'DHR', 'PM', 'NEE', 'VZ', 'INTC', 'RTX', 'HON', 'LOW', 'UPS', 'INTU', 'SPGI', 'NKE', 'COP', 'QCOM', 'BMY', 'CAT', 'UNP', 'BA', 'ISRG', 'GE', 'IBM', 'AMGN', 'AMAT', 'MDT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'BLK', 'GS', 'T', 'LMT', 'AXP', 'BKNG', 'SYK', 'ADI', 'TJX', 'ELV', 'MDLZ', 'GILD', 'ADP', 'MMC', 'C', 'AMT', 'CVS', 'VRTX', 'SCHW', 'LRCX', 'MO', 'TMUS', 'SLB', 'ETN', 'ZTS', 'CI', 'PYPL', 'FI', 'CB', 'SO', 'REGN', 'BSX', 'EQIX', 'BDX', 'PANW', 'DUK', 'EOG', 'MU', 'AON', 'ITW', 'CSX', 'SNPS', 'PGR', 'APD', 'KLAC', 'CME', 'NOC', 'CDNS', 'ICE', 'CL', 'SHW', 'WM', 'HCA', 'TGT', 'FCX', 'FDX', 'F', 'ORLY', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PNC', 'PH', 'MPC', 'APH', 'ROP', 'FTNT', 'MCHP', 'USB', 'CCI', 'MAR'

188

# SOME RESULTS, HISTORY LOG

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1.001, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
6056.55094263172 1011
Current optimal stock list len is 127

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
7049.139814422607 599
Current optimal stock list len is 99,

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 10, 'bull_trend_coef': 1.12}
8616.535972952845 423
Current optimal stock list len is 97

{'is_near_global_max_prt': 80, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.15, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
8455.558336365704 662
Current optimal stock list len is 71

v1:
{'is_near_global_max_prt': 80, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 9, 'bull_trend_coef': 1.12}
5762.099450752256 170 , number_red_candles= 6

24/08/2024
{'is_near_global_max_prt': 100, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.25, 'buy_ratio_border': 0, 'bull_trend_coef': 1.12}
119412.97672809978 380

In [100]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

  # if df['ha_pct'].iloc[i] > RIV \
  # and reverse_flag==False \
  # and sell_finished == True \
  # and last_top / df['open'].iloc[i] > last_top_ratio \
  # and i - last_top_i > distance_from_last_top \
  # and buy_time_cond \
  # and buy_ratio > buy_ratio_border \
  # and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

# Current_optimal_stock_list 
# stock_name_list_opt = ['JPM', 'UNH', 'AVGO', 'HD', 'PEP', 'COST', 'CRM', 'WMT',
#                                 'MCD', 'ABT', 'TXN', 'WFC', 'INTC', 'RTX', 'HON', 'SPGI',
#                                   'BA', 'AMGN', 'AMAT', 'SBUX', 'PLD', 'MS', 'DE', 'AXP', 
#                                   'ADI', 'TJX', 'MMC', 'CVS', 'SCHW', 'TMUS', 'ETN', 'CI', 
#                                   'BSX', 'AON', 'ITW', 'SNPS', 'KLAC', 'CDNS', 'WM', 'HCA',
#                                     'FDX', 'MMM', 'CMG', 'GM', 'MCK', 'NXPI', 'NSC',
#                                       'HUM', 'DXCM', 'PH', 'MPC', 'ROP', 'MCHP', 'MSI', 'GD',
#                                         'JCI', 'SRE', 'ADSK', 'AZO', 'TDG', 'AJG', 'TT', 'PCAR',
#                                           'COF', 'CTAS', 'AIG', 'ANET', 'SPG', 'MSCI', 'ROK']

stock_name_list_opt = [
'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
]

# stock_name_list_opt =['GOOG', 'JPM', 'XOM', 'UNH', 'JNJ', 'V', 'AVGO', 'PG', 'LLY', 'MA', 'HD', 'CVX', 'MRK', 'PEP', 'COST', 'ABBV', 'ADBE', 'KO', 'CRM', 'WMT', 'MCD', 'CSCO', 'BAC', 'PFE', 'TMO', 'ACN', 'NFLX', 'ABT', 'AMD', 'LIN', 'ORCL', 'CMCSA', 'TXN', 'DIS', 'WFC', 'DHR', 'PM', 'NEE', 'VZ', 'INTC', 'RTX', 'HON', 'LOW', 'UPS', 'INTU', 'SPGI', 'NKE', 'COP', 'QCOM', 'BMY', 'CAT', 'UNP', 'BA', 'ISRG', 'GE', 'IBM', 'AMGN', 'AMAT', 'MDT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'BLK', 'GS', 'T', 'LMT', 'AXP', 'BKNG', 'SYK', 'ADI', 'TJX', 'ELV', 'MDLZ', 'GILD', 'ADP', 'MMC', 'C', 'AMT', 'CVS', 'VRTX', 'SCHW', 'LRCX', 'MO', 'TMUS', 'SLB', 'ETN', 'ZTS', 'CI', 'PYPL', 'FI', 'CB', 'SO', 'REGN', 'BSX', 'EQIX', 'BDX', 'PANW', 'DUK', 'EOG', 'MU', 'AON', 'ITW', 'CSX', 'SNPS', 'PGR', 'APD', 'KLAC', 'CME', 'NOC', 'CDNS', 'ICE', 'CL', 'SHW', 'WM', 'HCA', 'TGT', 'FCX', 'FDX', 'F', 'ORLY', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PNC', 'PH', 'MPC', 'APH', 'ROP', 'FTNT', 'MCHP', 'USB', 'CCI', 'MAR', 'MSI', 'GD', 'PSA', 'JCI', 'PSX', 'SRE', 'ADSK', 'AZO', 'TDG', 'ECL', 'AJG', 'KMB', 'TEL', 'TT', 'AEP', 'EL', 'PCAR', 'OXY', 'TFC', 'CARR', 'D', 'IDXX', 'GIS', 'ON', 'COF', 'ADM', 'MNST', 'NUE', 'CTAS', 'AIG', 'EXC', 'VLO', 'MRNA', 'ANET', 'WMB', 'O', 'STZ', 'IQV', 'HLT', 'CHTR', 'WELL', 'BIIB', 'SPG', 'MSCI', 'DHI', 'ROK']


len(stock_name_list_opt), len(stock_name_list)


parameter is_near_global_max_prt best value is 96
parameter distance_from_last_top best value is 0
parameter last_top_ratio best value is 1.01
parameter RIV best value is 0.05
parameter buy_ratio_border best value is 0
parameter bull_trend_coef best value is 1.07


(110, 188)

In [101]:
len(stock_name_list), len(stock_name_list_opt)

(188, 110)

In [70]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
# print("TRAIN RESULTS")
# total_profit = 0
# total_loses = 0
# total_wins = 0
# for ticker in stock_name_list_opt:
#   file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
#   path = os.path.join(parent_path, folder_saved_models, file_name)
#   df = load_data_one_stock(path)
#   profit, wins, loses = one_stock_simulation(df, ticker, mode = 'train', print_sim_results=True, print_trans_history=False)
#   total_profit += profit
#   total_loses += loses
#   total_wins += wins
# print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / total_loses + 1:.2f}')

print("TEST RESULTS")
total_profit = 0
total_loses = 0
total_wins = 0
for ticker in stock_name_list_opt:
# for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  if df.shape[0] > 0:
    profit, wins, loses = one_stock_simulation(df, ticker, mode = 'test', print_sim_results=True, print_trans_history=False)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / (total_loses + 1):.2f}')
  else:
    print(f'{ticker} empty df')


TEST RESULTS
--------------------------------------------------
Simulation results for ticker GOOG:
{'number success': 1, 'number fails': 1, 'number wins': 2, 'proportion': 1.0, 'profit': np.float64(-5.248949981689453), 'transactions': [{'type': 'success', 'buy_price': np.float64(129.08), 'sell_price': np.float64(129.46), 'buy_time': Timestamp('2023-09-26 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-26 15:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': np.float64(133.83), 'sell_price': np.float64(133.32), 'buy_time': Timestamp('2023-10-03 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-10-03 15:30:00-0400', tz='America/New_York')}]}
Total Profit is -5.25, total loses is 1, total wins is 2, proportion: 1.00
--------------------------------------------------
Simulation results for ticker JPM:
{'number success': 0, 'number fails': 1, 'number wins': 1, 'proportion': 0.5, 'profit': np.float64(-6.696629859924316), 'transa

In [31]:
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 30, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.1, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 20, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 70, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.2, 'buy_ratio_border': 7, 'bull_trend_coef': 1.07}
best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.9, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07,
              'is_near_global_max_prt2': 60,  'global_max_prop_ratio' : 1.08, 'nrc': 6}
for param in best_params:
  vars()[param] = best_params[param]

stock_name_list_opt = ['MAR', 'MO', 'DHI', 'PYPL', 'HLT', 'BIIB', 'ZTS', 'TMUS', 'CMG', 'WELL', 'DUK',
                       'MCHP', 'MS', 'ROK', 'INTU', 'EXC', 'CI', 'STZ', 'NXPI', 'COF', 'DIS', 'EL', 'MU',
                       'NOW', 'CAT', 'HUM', 'OXY', 'PSX', 'COP', 'QCOM', 'ON', 'CVS', 'VLO', 'TJX', 'PGR',
                       'TXN', 'EOG', 'CSX', 'PM', 'MMM', 'ADM', 'GM', 'PEP', 'VZ', 'REGN', 'PCAR', 'GILD',
                       'EW', 'ANET', 'GIS', 'LIN', 'TEL', 'VRTX', 'BLK', 'ELV', 'KO', 'DE', 'KLAC', 'SBUX',
                       'PLD', 'PANW', 'CHTR', 'SNPS', 'MCK', 'TFC', 'USB', 'TDG', 'PFE', 'MRK', 'CB', 'MPC',
                       'GOOG', 'TT', 'PSA', 'CME', 'AIG', 'IDXX', 'BSX', 'WFC', 'ABT', 'TGT', 'EQIX', 'CTAS',
                       'BA', 'AON', 'SYK', 'F', 'SRE', 'JCI', 'NKE', 'MCD', 'TMO', 'SO', 'WMT', 'UNH', 'CSCO',
                      'CRM', 'PG', 'JNJ', 'HCA', 'XOM', 'ETN', 'V', 'COST', 'GS', 'FCX', 'MDT', 'NEE', 'CL',
                       'BMY', 'SPG', 'ADI', 'ABBV', 'WM', 'NUE', 'MDLZ', 'MA', 'MRNA', 'ROP', 'AEP', 'IBM',
                      'EMR', 'JPM', 'APD', 'AMD', 'T', 'INTC', 'ITW', 'AMGN', 'ADSK', 'SHW', 'NOC', 'NSC', 'SCHW', 'ICE']

In [247]:
# best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV1': 0.3, 'RIV2': 0.9, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07,
#               'is_near_global_max_prt2': 60,  'global_max_prop_ratio' : 1.08, 'nrc': 3, 'tp1': 0, 'tp2': 0, 'tp': 1}
# for param in best_params:
#   vars()[param] = best_params[param]

# best_params = {'tp1': 0.005, 'tp2': 0.04, 'tp3': 0.94, 'tp4': 1.006, 'tp5' : 1.0015, 'tp6' : 1.006}
# best_params = {'tp1': np.float64(0.0), 'tp2': np.float64(0.045000000000000005), 'tp4': np.float64(1.008999999999999), 'tp5': 1.0015, 'tp6': np.float64(1.0029999999999997)}
# best_params ={'tp2': np.float64(0.08500000000000002), 'tp4': np.float64(1.002), 'tp5': 1.0015, 'tp6': np.float64(1.003)}
# best_params = {'tp1': np.float64(0.009), 'tp2': np.float64(0.085), 'tp3': np.float64(0.95), 
#                'tp4': np.float64(1.002), 'tp5': 1.0015, 'tp6': np.float64(1.003)}
best_params = {'tp2': np.float64(0.07500000000000001), 'tp4': np.float64(1.0039999999999998), 'tp5': 1.0015, 'tp6': np.float64(1.005)}

for param in best_params:
  vars()[param] = best_params[param]

In [248]:
print("FULL RESULTS OPTIMIZATION")
# parameters:
mode = 'test'
lose_coef = 0.995
gain_coef = 1.005
# gain_coef = 1.02
use_optimal_stock_list = False
use_stock_name_list_selected = False
total_profit = 0
total_loses = 0
total_wins = 0

# tp1 = np.float64(0.21)
# tp2 =  np.float64(1.01)
tp = 0.3

if use_optimal_stock_list:
  current_stock_name_list = stock_name_list_opt[:]
else:
  current_stock_name_list = stock_name_list[:]

if use_stock_name_list_selected:
  current_stock_name_list = stock_name_list_selected[:]

# current_stock_name_list = ['CME', 'AMT']
# current_stock_name_list = ['AMD', 'JPM']
current_stock_name_list = ['ON']
# current_stock_name_list = ['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD', 'ORCL', 'TXN', 'WFC', 'INTC', 'INTU', 'QCOM', 'BMY', 'CAT', 'BA', 'ISRG', 'GE', 'AMAT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'GS', 'AXP', 'ADI', 'C', 'CVS', 'SCHW', 'LRCX', 'SLB', 'ETN', 'PYPL', 'PANW', 'MU', 'SNPS', 'APD', 'KLAC', 'CDNS', 'TGT', 'FCX', 'F', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PH', 'MPC', 'APH', 'FTNT', 'MCHP', 'USB', 'JCI', 'PSX', 'ADSK', 'TDG', 'TT', 'EL', 'PCAR', 'OXY', 'TFC', 'CARR', 'IDXX', 'ON', 'COF', 'MNST', 'NUE', 'AIG', 'VLO', 'MRNA', 'ANET', 'IQV', 'CHTR', 'BIIB', 'DHI', 'ROK']
# current_stock_name_list = ['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD']
# current_stock_name_list = ['DHI', 'CDNS', 'ORCL', 'CMG', 'ADBE', 'APH', 'AMD', 'NOW', 'EMR', 'ETN', 'EW', 'AMAT', 'FTNT', 'AIG', 'AVGO', 'TT', 'DE', 'GM', 'CAT', 'ON', 'INTU', 'LRCX', 'PYPL', 'LLY', 'GOOG', 'VLO', 'C', 'BAC', 'PH']
# current_stock_name_list  = ['ABT', 'LIN', 'MDT']
# current_stock_name_list  = ['ABT', 'LIN', 'MDT', 'NFLX', 'XOM', 'UNH', 'PG', 'MA', 'JNJ', 'V', 'CRM', 'CVX', 'PEP', 'MRK', 'ABBV',
#                         'ADBE', 'KO', 'COST', 'PFE', 'WMT', 'MCD', 'CSCO', 'ACN', 'TMO', 'HD', 'JPM', 'BIIB', 'WELL', 'CTAS',
#                         'AIG', 'ADM', 'O', 'KMB', 'TT', 'ADSK', 'UNP', 'COP', 'ISRG', 'IBM', 'GE', 'AMGN', 'SPGI', 'DHR', 
#                         'PM', 'INTU', 'UPS', 'VZ', 'LOW', 'HON', 'RTX', 'CMCSA', 'ORCL', 'NEE', 'DIS', 'MDLZ', 'ELV', 'T']

df_opt = pd.DataFrame(columns=['ticker','profit', 'wins', 'loses', 'proportion'])

for ticker in current_stock_name_list:
# for ticker in ['GOOG']:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)

  # if ticker in opt_stocks_for_bear_trend:
  #   gain_coef = 1.02
  # else:
  #   gain_coef = 1.005

  if df.shape[0] > 0:
    try:
      profit, wins, loses = one_stock_simulation(df, ticker,
                                                mode=mode,
                                                _gain_coef=gain_coef,
                                                _lose_coef=lose_coef, 
                                                print_sim_results=True, 
                                                print_trans_history=True)
    except:
      profit, wins, loses = 0 ,0 ,0
    if loses == 0:
      proportion = wins
    else:
      proportion = wins / loses
    df2 = pd.DataFrame({'ticker': [ticker],'profit': [profit], 'wins': [wins], 'loses': [loses], 'proportion': [proportion]})
    df_opt = pd.concat([df_opt, df2], ignore_index=True)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    if total_loses == 0:
      total_proportion = total_wins
    else:
      total_proportion = total_wins / total_loses
  else:
    print(f'{ticker} empty df')
c.print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_proportion:.2f}', color = 'blue')
c.print('Parameters are:', color = 'blue' )
c.print(f'gain_coef is {gain_coef}; lose_coef is {lose_coef}; use_optimal_stock_list is {use_optimal_stock_list}, use_selected_optimal_stock_list is \
{use_stock_name_list_selected} mode is {mode}', color = 'blue')
for param in best_params:
   print(f'{param} is {vars()[param]}')

FULL RESULTS OPTIMIZATION
--------------------------------------------------
Simulation results for ticker ON:
amp_7 is 1.0452001084363092, amp_30 is 1.0977517828317962, amp_100 is 1.168012631732455,  
{'number success': 37, 'number fails': 36, 'number wins': 37, 'proportion': 1.0, 'profit': np.float64(-132.94165006260556), 'transactions': [{'type': 'fail', 'buy_price': np.float64(96.35), 'sell_price': np.float64(95.87), 'buy_time': Timestamp('2023-09-14 15:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-15 09:30:00-0400', tz='America/New_York')}, {'type': 'fail', 'buy_price': np.float64(93.46), 'sell_price': np.float64(92.99), 'buy_time': Timestamp('2023-09-15 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-15 13:30:00-0400', tz='America/New_York')}, {'type': 'fail', 'buy_price': np.float64(93.61), 'sell_price': np.float64(93.14), 'buy_time': Timestamp('2023-09-15 13:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-15 14

C:\Users\IvanSpok\AppData\Local\Temp\ipykernel_16980\3711337719.py:65: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_opt = pd.concat([df_opt, df2], ignore_index=True)


Results:
1. use ordered selected stock list;
2. Use danamic gain coeficient'


In [189]:
df_opt = df_opt.sort_values(by='profit', ascending=False)
print(df_opt)
df_opt.to_csv('df_opt.csv')

    ticker      profit wins loses proportion
185    DHI  122.691973   37    14   2.642857
82    LRCX  103.170779   32    12   2.666667
121    CMG   87.467033   14     2        7.0
123     GM   66.900268   40    21   1.904762
116    TGT   62.808471   28    13   2.153846
..     ...         ...  ...   ...        ...
49     BMY  -74.872181   16    17   0.941176
120    MMM  -79.410499   11    14   0.785714
173   MRNA  -82.957845   68    52   1.307692
137   MCHP  -98.021634   34    31   1.096774
100     MU -103.039634   39    35   1.114286

[187 rows x 5 columns]


In [200]:
stock_name_list_selected = df_opt.loc[df_opt['profit'] == 0].sort_values(by='profit', ascending=False)['ticker']
print(stock_name_list_selected.to_list())
print(len(stock_name_list_selected))
# stock_name_for_check = [
# 'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
# 'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
# 'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
# 'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
# 'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
# 'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
# 'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
# 'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
# 'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
# ]
# (stock_name_for_check == stock_name_list_selected).all()
opt_stocks_for_bear_trend = ['BA', 'INTU', 'MCHP', 'LLY', 'DHI', 'ANET', 'AIG', 'NUE', 'MAR', 'OXY', 'ON',
  'GE', 'AMAT', 'NXPI', 'SNPS', 'UNP', 'KLAC', 'BSX', 'MSI', 'CRM', 'CAT', 'ADI',
    'ETN', 'JCI', 'HLT', 'CSCO', 'WMT', 'TDG', 'TT', 'ECL', 'LOW', 'ADSK', 'TJX',
    'VRTX', 'APH', 'ABBV', 'STZ', 'SBUX', 'DE', 'MRK', 'CTAS', 'MNST', 'CME', 'MO', 'TXN', 'ITW']
# print(stock_name_list_selected)
# df_opt.loc[df_opt['profit'] > 0].sort_values(by='profit', ascending=False)


['ABT', 'LIN', 'MDT', 'NFLX', 'XOM', 'UNH', 'PG', 'MA', 'JNJ', 'V', 'CRM', 'CVX', 'PEP', 'MRK', 'ABBV', 'ADBE', 'KO', 'COST', 'PFE', 'WMT', 'MCD', 'CSCO', 'ACN', 'TMO', 'HD', 'JPM', 'BIIB', 'WELL', 'CTAS', 'AIG', 'ADM', 'O', 'KMB', 'TT', 'ADSK', 'UNP', 'COP', 'ISRG', 'IBM', 'GE', 'AMGN', 'SPGI', 'DHR', 'PM', 'INTU', 'UPS', 'VZ', 'LOW', 'HON', 'RTX', 'CMCSA', 'ORCL', 'NEE', 'DIS', 'MDLZ', 'ELV', 'T', 'SYK', 'FDX', 'WM', 'PGR', 'APD', 'SHW', 'CME', 'REGN', 'SO', 'EOG', 'AON', 'ITW', 'CSX', 'BSX', 'EQIX', 'BDX', 'PLD', 'FI', 'MO', 'TMUS', 'MMC', 'AMT', 'CVS', 'VRTX', 'ADP', 'CI', 'ZTS', 'CB', 'TJX', 'GILD', 'BLK', 'AXP', 'LMT', 'MSCI', 'SPG', 'STZ', 'HLT', 'WMB', 'EXC', 'D', 'AEP', 'MNST', 'SRE', 'GIS', 'IDXX', 'ICE', 'NOC', 'HCA', 'CL', 'DUK', 'AZO', 'AJG', 'ECL', 'TEL', 'PSA', 'TDG', 'NSC', 'ROP', 'MPC', 'MCO', 'PSX', 'MSI', 'GD', 'CCI', 'MCK', 'OXY']
123


In [120]:
df_opt_bear_trend = df_opt.sort_values(by='proportion', ascending=False)

df_opt_bear_trend = df_opt_bear_trend.loc[df_opt_bear_trend['proportion'] >= 3].sort_values(by='profit', ascending=False)
print(df_opt_bear_trend)
print(df_opt_bear_trend['ticker'].to_list())

    ticker      profit wins loses  proportion
0       BA  674.602803   38     6    6.333333
8     INTU  657.527402   40     7    5.714286
6     MCHP  599.032862   41     8    5.125000
13     LLY  587.327168   26     4    6.500000
2      DHI  573.843221   36     8    4.500000
17    ANET  563.029703   36    12    3.000000
11     AIG  491.698669   32     7    4.571429
24     NUE  465.124099   39    12    3.250000
31     MAR  456.696729   33     6    5.500000
1      OXY  455.211293   36    10    3.600000
3       ON  444.715145   45    13    3.461538
21      GE  440.864404   28     5    5.600000
15    AMAT  410.183924   36    11    3.272727
10    NXPI  404.862433   38    10    3.800000
28    SNPS  376.630274   33    10    3.300000
87     UNP  360.880210   23     6    3.833333
30    KLAC  358.535270   32     8    4.000000
7      BSX  349.135627   20     4    5.000000
61     MSI  348.189323   26     5    5.200000
85     CRM  330.056228   33    11    3.000000
43     CAT  329.017333   26     7 